### ETL Step 1 --> Installations, Imports, Setting up credentials

In [ ]:
pip install cryptowatch-client

In [ ]:
from cryptowatch_client import Client
client = Client()

import numpy as np
import pandas as pd
from datetime import datetime
import fnmatch
import re
import os
import requests
import json
import math
import random
from timeit import default_timer as timer
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### ETL Step 2 --> Getting all active markets

In [ ]:
df_markets = pd.DataFrame(client.get_markets().json()['result']) # 19,343 markets (exchange + pair)
df_markets_active = df_markets[df_markets['active']==True] # 14,486
#df_markets_active

### ETL Step 3 --> Getting USDT Price data for key assets excluding stablecoins and Fiat

In [ ]:
key_assets = ['btc','eth','sol','ada','doge','usdt']
key_assets_usdt_market= pd.DataFrame()
for ele in key_assets[-1:]:
  data = df_markets_active[(df_markets_active['pair'].str.startswith(ele)) & (df_markets_active['pair'].str.endswith('usdt')) & ((df_markets_active['pair'].str.len() - len(ele) == 4) ) ]
  key_assets_usdt_market= key_assets_usdt_market.append(data)
#list(key_assets_usdt_market.pair.unique())

In [ ]:
def get_markets_ohlc(exchange,pair):
    uri = 'https://api.cryptowat.ch/markets/{0}/{1}/ohlc'.format(exchange,pair)
    response = requests.get(uri, headers={'X-CW-API-Key' : os.environ.get("X-CW-API-Key")]})
    statuscode = response.status_code    
    return response,statuscode    

In [ ]:
#exchanges       = df_markets_active['exchange'].unique().tolist()
exchanges        = ['binance','ftx','coinbase-pro','kraken','kucoin']
assets_pairs     = list(key_assets_usdt_market.pair.unique())
periods          = '86400' # daily  

df_results_price= pd.DataFrame()
for exchange in exchanges:
    for assets_pair in assets_pairs:
        if  get_markets_ohlc(exchange=exchange,pair=assets_pair)[1] == 200: 
            df = pd.DataFrame(get_markets_ohlc(exchange=exchange,pair=assets_pair)[0].json()['result']['86400'])
            df = df.rename(columns={0: "CloseTime", 1: "OpenPrice", 2: "HighPrice",3: "LowPrice", 4: "ClosePrice", 5: "quote_volume", 6: "base_volume"})
            df['exchange']= exchange
            df['key_asset']    = assets_pair.replace('usdt', '')
            df_results_price = df_results_price.append(df) 
df_results_price # 18898 rows

In [ ]:
df_results_price = df_results_price.drop(['OpenPrice', 'HighPrice','LowPrice','quote_volume','base_volume'], axis=1)   
df_results_price['CloseTime'] = pd.to_datetime(df_results_price['CloseTime'], unit='s')
df_results_price['exchange'] = df_results_price['exchange'].astype(pd.StringDtype())
df_results_price['key_asset'] = df_results_price['key_asset'].astype(pd.StringDtype())
df_results_price = df_results_price.rename(columns={"CloseTime": "day","ClosePrice": "price_USDT", "quote_asset": "key_asset"})
df_results_price # 18898  rows

### ETL Step 4 --> Getting Volume data for all key assets

In [ ]:
unique_pairs = list(df_markets_active['pair'].unique()) # 7847
key_assets_pair_non_unique = []
for asset in key_assets:
    for pair in unique_pairs:
        if pair.startswith(asset):
            key_assets_pair_non_unique.append(pair) 
        if  pair.endswith(asset)  and not pair.endswith('weth') and  not pair.endswith('wbtc') : 
            key_assets_pair_non_unique.append(pair)
            
key_assets_pair = list(set(key_assets_pair_non_unique))   
key_assets_pair = [item for item in key_assets_pair if len(item) < 9]
len(key_assets_pair) # 3232
#random.sample(key_assets_pair, 20)

In [ ]:
start = timer()

#exchanges       = df_markets_active['exchange'].unique().tolist() 
exchanges        = ['binance','ftx','coinbase-pro','kraken','kucoin']
#assets_pairs    = list(df_markets_active['pair'].unique())
assets_pairs     = key_assets_pair
periods          = '86400' # daily 

df_results_volume= pd.DataFrame()
for exchange in exchanges:
    for assets_pair in assets_pairs:
        try :
            if  get_markets_ohlc(exchange=exchange,pair=assets_pair)[1] == 200: 
                df = pd.DataFrame(get_markets_ohlc(exchange=exchange,pair=assets_pair)[0].json()['result']['86400']) 
                df = df.rename(columns={0: "CloseTime", 1: "OpenPrice", 2: "HighPrice",3: "LowPrice", 4: "ClosePrice", 5: "quote_volume", 6: "base_volume"})
                df['exchange']= exchange
                df['market']    = assets_pair
                df_results_volume = df_results_volume.append(df) 
        except keyError:        

end = timer()
print(end - start) # ~30 mins

In [ ]:
#df_results_volume # 1,572,864

In [ ]:
## Data Transformations
df_results_volume = df_results_volume.drop(['OpenPrice', 'HighPrice','LowPrice'], axis=1)   
df_results_volume['CloseTime'] = pd.to_datetime(df_results_volume['CloseTime'], unit='s')
df_results_volume['exchange'] = df_results_volume['exchange'].astype(pd.StringDtype())
df_results_volume['market'] = df_results_volume['market'].astype(pd.StringDtype())
#df_results_volume 

In [ ]:
df_results_volume['market'].unique()
## Need to check all the market with key asset (because startswith and endswith will not satisfy all pairs)

In [20]:
### Quote [daily total volume and average close price]
df_quote = pd.DataFrame()
for key_asset in  key_assets :
    df = df_results_volume.loc[(df_results_volume['market'].str.startswith(key_asset))].groupby(['exchange','CloseTime']).agg({'quote_volume':"sum", 'ClosePrice': "mean"})
    df['quote_asset'] = key_asset
    df_quote = df_quote.append(df) 
df_quote=df_quote.reset_index()
df_quote= df_quote.rename(columns={"quote_volume": "total_volume_quote", "ClosePrice": "avg_ClosePrice_quote"})

### Base [daily total volume and average close price]
df_base = pd.DataFrame()
for key_asset in  key_assets :
    df  = df_results_volume.loc[(df_results_volume['market'].str.endswith(key_asset))].groupby(['exchange','CloseTime']).agg({'base_volume': "sum", 'ClosePrice': "mean"})
    df['base_asset'] = key_asset
    df_base = df_base.append(df) 
df_base=df_base.reset_index()
df_base= df_base.rename(columns={"base_volume": "total_volume_base", "ClosePrice": "avg_ClosePrice_base"})

In [ ]:
#df_quote #  37,007
#df_base #  22,383

In [23]:
# results
df_quote['quote_asset'].unique(),df_base['base_asset'].unique()

(array(['btc', 'eth', 'sol', 'ada', 'doge', 'usdt'], dtype=object),
 array(['btc', 'eth', 'doge', 'usdt'], dtype=object))

In [ ]:
### Volume Summary
df_volume = pd.merge( df_base,df_quote,  how='outer', left_on=['exchange','CloseTime','base_asset'], right_on = ['exchange','CloseTime','quote_asset'])
df_volume['base_asset'] = df_volume['base_asset'].astype(pd.StringDtype())
df_volume['quote_asset'] = df_volume['quote_asset'].astype(pd.StringDtype())
df_volume['exchange'] = df_volume['exchange'].astype(pd.StringDtype())
#df_volume # 38404

In [25]:
# check
print(list(df_quote['quote_asset'].unique()), list(df_base['base_asset'].unique()), list(df_volume['quote_asset'].unique()), list(df_volume['base_asset'].unique()))

['btc', 'eth', 'sol', 'ada', 'doge', 'usdt'] ['btc', 'eth', 'doge', 'usdt'] [<NA>, 'btc', 'eth', 'doge', 'usdt', 'sol', 'ada'] ['btc', 'eth', 'doge', 'usdt', <NA>]


In [ ]:
## Removing the Null values with two seperate base/quote dfs and merging them again to get final result of volume df
df_base_vol  = df_volume[df_volume['base_asset']!= 'NaN'].groupby(['exchange','CloseTime','base_asset'])/
                     .agg({'total_volume_base': "sum"})
df_base_vol = df_base_vol.reset_index()
df_base_vol = df_base_vol.rename(columns={"CloseTime": "day","total_volume_base": "total_daily_volume", "base_asset": "key_asset"})

df_quote_vol = df_volume[df_volume['quote_asset']!= 'NaN'].groupby(['exchange','CloseTime','quote_asset'])/
                              .agg({'total_volume_quote': "sum"})
df_quote_vol = df_quote_vol.reset_index()
df_quote_vol = df_quote_vol.rename(columns={"CloseTime": "day","total_volume_quote": "total_daily_volume", "quote_asset": "key_asset"})

df_results_volume_final = df_base_vol.append(df_quote_vol) 
df_results_volume_final = df_results_volume_final.groupby(['exchange','day','key_asset'])/
                              .agg({'total_daily_volume': "sum"})
df_results_volume_final=df_results_volume_final.reset_index()
#df_results_volume_final # 38,404

### ETL Step 5 --> Price & Volume summary

In [ ]:
df_price_volume = df_results_volume_final.merge(df_results_price,  on=['exchange','day','key_asset'],how='inner' )
df_price_volume['key_asset'] = df_price_volume['key_asset'].astype(pd.StringDtype())
df_price_volume['exchange'] = df_price_volume['exchange'].astype(pd.StringDtype())
#df_price_volume # 18,898

In [ ]:
df_price_volume.dtypes

### Analysis 1  --> Total CEX Daily Volume

In [306]:
Exchange_daily_volume  = df_price_volume.assign(total_daily_volume_in_usd = df_price_volume['price_USDT']*df_price_volume['total_daily_volume']).groupby(['exchange','day'])\
                                .agg({'total_daily_volume_in_usd':sum})\
                                .rename(columns={'total_daily_volume_in_usd':'volume'}) \
                                .reset_index()
Exchange_daily_volume['exchange'] = Exchange_daily_volume['exchange'].astype(pd.StringDtype())     

dash_df = Exchange_daily_volume.query("day>='2020-01-01'") # Only visualising data starting on JAN 2020 for simplicity 
Dash_one = px.line(dash_df, x="day", 
                   y="volume",
                   title="Total Daily Volume of Key Assets (BTC,ETH,SOL,ADA and DOGE)",
                   color='exchange')
Dash_one.update_layout( xaxis_title='Day', yaxis_title='Daily Volume in USD')
Dash_one.show(renderer="colab")

### Analysis 2--> Volume Share between top CEXs

In [305]:
#exchanges        = ['binance','ftx','coinbase-pro','kraken','kucoin']
#Exchange_daily_volume.head(2)

#for exchange in exchanges:
#Daily_volume = Exchange_daily_volume
Daily_volume = Exchange_daily_volume.groupby('day').agg({'volume': 'sum'}).rename(columns={'volume':'daily_volume'})

Percentage_Daily_volume = pd.merge( Exchange_daily_volume,Daily_volume,  how='left', left_on=['day'], right_on = ['day'])
# dfv=Percentage_Daily_volume.query("day=='2020-01-01'")
# dfv
Percentage_Daily_volume['Volume_Share'] = Percentage_Daily_volume['volume']/Percentage_Daily_volume['daily_volume']
# #Percentage_Daily_volume['percentage_vol'] = Percentage_Daily_volume['percentage_vol'].map("{:.2%}".format)
#Kraken_Percentage_Daily_volume=Kraken_Percentage_Daily_volume.reset_index()
Percentage_Daily_volume.dtypes

dash_df = Percentage_Daily_volume.query("day>='2020-01-01'") 
Dash_two = px.line(dash_df, x="day", 
                   y="Volume_Share",
                   title="Daily Volume Share between Kraken and it's competitors (Binance,FTX,Coinbase Pro,KuCoin)",
                   color='exchange')
Dash_two.update_layout(yaxis_title='Volume Share')
Dash_two.show(renderer="colab")

### Analysis 3 --> Trends in Volume and Prices (Key Assets only)

In [310]:
Asset_daily_volume_price  = df_price_volume.assign(total_daily_volume_in_usd = df_price_volume['price_USDT']*df_price_volume['total_daily_volume']).groupby(['key_asset','day'])\
                                  .agg({'total_daily_volume_in_usd':sum,'total_daily_volume':sum})\
                                  .reset_index()
Asset_daily_volume_price['Weighted_avg_Price'] = Asset_daily_volume_price['total_daily_volume_in_usd'] /Asset_daily_volume_price['total_daily_volume'] 
Asset_daily_volume_price['key_asset'] = Asset_daily_volume_price['key_asset'].astype(pd.StringDtype())
Asset_daily_volume_price['total_daily_volume_in_usd'] = Asset_daily_volume_price['total_daily_volume_in_usd'].astype('int64')
#Asset_daily_volume_price.head(2)       

dash_df = Asset_daily_volume_price.query("day>='2020-01-01' and key_asset =='btc'")

# Create figure with secondary y-axis
Dash_three_a = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
Dash_three_a.add_trace(go.Scatter(x=dash_df['day'], y=dash_df['Weighted_avg_Price'], name="Price"),secondary_y=False)
Dash_three_a.add_trace(go.Scatter(x=dash_df['day'], y=dash_df['total_daily_volume_in_usd'], name="Volume in $USD"),secondary_y=True)

# Add figure title
Dash_three_a.update_layout(title_text="<b>BTC</b> (Daily Volume Vs Aeverage Price)")

# Set y-axes titles
Dash_three_a.update_yaxes(title_text=" Weighted Aeverage Price in $USD", secondary_y=False)
Dash_three_a.update_yaxes(title_text=" Volume in USD", secondary_y=True)

Dash_three_a.show(renderer="colab")

In [271]:
dash_df = dash_df = Asset_daily_volume_price.query("day>='2020-01-01' and key_asset not in ('btc')")
dash_eth = dash_df[(dash_df['key_asset']=='eth')]
dash_sol = dash_df[(dash_df['key_asset']=='sol')]
dash_ada = dash_df[(dash_df['key_asset']=='ada')]
dash_doge = dash_df[(dash_df['key_asset']=='doge')]

Dash_three_b = make_subplots(rows=2, cols=2,
                    specs=[[{"secondary_y": True}, {"secondary_y": True}],
                           [{"secondary_y": True}, {"secondary_y": True}]],
                    subplot_titles=('<b>ETH<b>', '<b>SOL<b>', '<b>ADA<b>','<b>DOGE<b>'))

# plotly fig setup
fig = make_subplots(rows=1,cols=2,subplot_titles=('Subplot title1',  'Subplot title2')) 

# Top left
Dash_three_b.add_trace(go.Scatter(x=dash_eth['day'], y=dash_eth['Weighted_avg_Price'], name="ETH Price" ),row=1, col=1, secondary_y=False)
Dash_three_b.add_trace(go.Scatter(x=dash_eth['day'], y=dash_eth['total_daily_volume_in_usd'], name="ETH Volume" ),row=1, col=1, secondary_y=True)

# Top right
Dash_three_b.add_trace(go.Scatter(x=dash_sol['day'], y=dash_sol['Weighted_avg_Price'], name="SOL Price"),row=1, col=2, secondary_y=False)
Dash_three_b.add_trace(go.Scatter(x=dash_sol['day'], y=dash_sol['total_daily_volume_in_usd'], name="SOL Volume"),row=1, col=2, secondary_y=True)

# Bottom left
Dash_three_b.add_trace(go.Scatter(x=dash_ada['day'], y=dash_ada['Weighted_avg_Price'], name="ADA Price"),row=2, col=1, secondary_y=False)
Dash_three_b.add_trace(go.Scatter(x=dash_ada['day'], y=dash_ada['total_daily_volume_in_usd'], name="ADA Volume"),row=2, col=1, secondary_y=True)

# Bottom right
Dash_three_b.add_trace(go.Scatter(x=dash_doge['day'], y=dash_doge['Weighted_avg_Price'], name="DOGE Price"),row=2, col=2, secondary_y=False)
Dash_three_b.add_trace(go.Scatter(x=dash_doge['day'], y=dash_doge['total_daily_volume_in_usd'], name="DOGE Volume"),row=2, col=2, secondary_y=True)

# Add figure title
Dash_three_b.update_layout(title_text="<b>Daily Volume Vs Aeverage Price<b>")

# Set y-axes titles
dash_three.update_yaxes(title_text="Price", secondary_y=False)
dash_three.update_yaxes(title_text="Volume", secondary_y=True)

dash_three.show(renderer="colab")